# Warehouse Woes

In [1]:
EMPTY = "."
FENCE = "#"
ROBOT = "@"
BOX = "O"
GPS_DISTANCE = 100

def get_matrix(area: str):
    return list(map(lambda line: list(line), area.split("\n")))

def render_matrix(matrix):
    return "\n".join(["".join(line) for line in matrix])

def parse(data: str):
    warehouse, directions = data.strip().split("\n\n")
    matrix = get_matrix(warehouse)
    moves = [move for line in directions.split("\n") for move in line]
    return matrix, moves
    
def get_robot_location(matrix, robot=ROBOT):
    for y in range(len(matrix)):
        try:
            x = matrix[y].index(robot)
            return (x, y)
        except ValueError:
            continue


def move_block(matrix, robots_position, dx, dy):
    x, y = robots_position
    current_block = matrix[y][x]
    next_block = matrix[y+dy][x+dx]

    if next_block == FENCE:
        return matrix

    if next_block == BOX:
       matrix = move_block(matrix, (x+dx, y+dy), dx, dy)

    next_block = matrix[y+dy][x+dx]
    if next_block == EMPTY:
        matrix[y+dy][x+dx] = current_block
        matrix[y][x] = EMPTY
        return matrix

    return matrix

def move_robot(matrix, direction):
    robot_position = get_robot_location(matrix)

    mapping = {
        "^": (0, -1),
        ">": (1, 0),
        "v": (0, 1),
        "<": (-1, 0),
    }

    return move_block(matrix, robot_position, *mapping[direction])

def get_box_locations(matrix, box=BOX):
    out = []
    for y in range(len(matrix)):
        for x in range(len(matrix[y])):
            if matrix[y][x] == box:
                out.append((x, y))

    return out

def get_gps_coordinates(boxes):
    return [box[0] + box[1] * GPS_DISTANCE for box in boxes]

with open("input.txt", "r") as f:
    data = f.read()

matrix, moves = parse(data)

for direction in moves:
    matrix = move_robot(matrix, direction)
    
boxes = get_box_locations(matrix)
sum(get_gps_coordinates(boxes))
# print(render_matrix(matrix))

1457740

Correct: `1457740`

In [2]:
BOX_LEFT = "["
BOX_RIGHT = "]"


def widen_matrix(matrix):
    out = []
    for y in range(len(matrix)):
        horizontal = []
        for x in range(len(matrix[y])):
            obj = matrix[y][x]
            if obj == FENCE:
                horizontal.extend([FENCE, FENCE])
            if obj == BOX:
                horizontal.extend([BOX_LEFT, BOX_RIGHT])
            if obj == EMPTY:
                horizontal.extend([EMPTY, EMPTY])
            if obj == ROBOT:
                horizontal.extend([ROBOT, EMPTY])
        out.append([*horizontal])
    return out

def can_move(matrix, robots_position, dx, dy):
    x, y = robots_position
    next_block = matrix[y+dy][x+dx]

    if next_block == FENCE:
        return False

    if next_block == BOX_LEFT:
        box_left = (x, y+dy)
        box_right = (x+1, y+dy)
        return can_move(matrix, box_left, dx, dy) and can_move(matrix, box_right, dx, dy)

    if next_block == BOX_RIGHT:
        box_left = (x, y+dy)
        box_right = (x-1, y+dy)
        return can_move(matrix, box_left, dx, dy) and can_move(matrix, box_right, dx, dy)

    if next_block == EMPTY:
        return True

    return False

def move_wide_blocks(matrix, robots_position, dx, dy):
    x, y = robots_position
    current_block = matrix[y][x]
    next_block = matrix[y+dy][x+dx]

    if next_block == FENCE:
        return matrix

    # horizontal movement
    if dx != 0:
        if next_block in [BOX_LEFT, BOX_RIGHT]:
           matrix = move_wide_blocks(matrix, (x+dx, y+dy), dx, dy)

        next_block = matrix[y+dy][x+dx]
        if next_block == EMPTY:
            matrix[y+dy][x+dx] = current_block
            matrix[y][x] = EMPTY
            return matrix

    # vertical movement
    if dy != 0 and can_move(matrix, robots_position, dx, dy):
        if next_block == BOX_LEFT:
            move_wide_blocks(matrix, (x, y+dy), dx, dy)
            move_wide_blocks(matrix, (x+1, y+dy), dx, dy)

        if next_block == BOX_RIGHT:
            move_wide_blocks(matrix, (x, y+dy), dx, dy)
            move_wide_blocks(matrix, (x-1, y+dy), dx, dy)
        
        next_block = matrix[y+dy][x+dx]
        if next_block == EMPTY:
            matrix[y+dy][x+dx] = current_block
            matrix[y][x] = EMPTY
            return matrix

    return matrix

def move_robot_wide(matrix, direction):
    robot_position = get_robot_location(matrix)

    mapping = {
        "^": (0, -1),
        ">": (1, 0),
        "v": (0, 1),
        "<": (-1, 0),
    }

    return move_wide_blocks(matrix, robot_position, *mapping[direction])

with open("input.txt", "r") as f:
    data = f.read()

matrix, moves = parse(data)
matrix = widen_matrix(matrix)


for direction in moves:
    matrix = move_robot_wide(matrix, direction)
    # print(direction)
    # print(render_matrix(matrix))
    # print()
# print(render_matrix(matrix))

boxes = get_box_locations(matrix, box=BOX_LEFT)
sum(get_gps_coordinates(boxes))

1467145

Correct: `1467145`